## Надо посчитать

1. Для каждого пользователя число таких сделок, у которых с момента открытия до
момента полного закрытия прошло меньше одной минуты

2. Для каждого пользователя найти число таких пар сделок, которые удовлетворяют
следующим условиям:
- Сделки совершены этим пользователем
- Разница между временем их открытия не более 30 секунд
- Направление этих сделок – противоположное (одна на покупку, другая на
продажу)

3. Мысленно разобьем все время на равные промежутки по 30 секунд. Найти среди всех
возможных пар пользователей такие, у которых более 10 сделок удовлетворяют
условиям:
- Открытия сделок попали в один промежуток времени
- Они открыты по одному и тому же инструменту
- Они принадлежат разным пользователям (одна одному пользователю из пары,
другая другому)
- Они открыты в разных направлениях (одна на покупку, другая на продажу)

## Результат

Скрипт на питоне, который создает:
1. csv файл в котором каждому логину соответствует 1 строка. Колонки – login, метрика из
пункта 1 и метрика из пункта 2
2. csv файл в котором перечислены все пары из пункта 3

In [1]:
import pandas as pd
import toml
from sqlalchemy import create_engine

# импорт для визуализации графовых данных
from jaal import Jaal

c:\Users\user\Anaconda3\lib\site-packages\dash_bootstrap_components\_table.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [2]:
config = toml.load('config.toml')

PG_HOST = config['host']
PG_PORT = config['port']
PG_USER = config['user']
PG_PASSWORD = config['password']

In [3]:
URL = f"postgresql://{PG_USER}:{PG_PASSWORD}@{PG_HOST}:{PG_PORT }/postgres"

In [4]:
engine = create_engine(URL)

### 1. Приведем данные из MT4 и MT5 к единому формату

Данные будут использовать несколько раз, поэтому создадим временную таблицу

In [5]:
engine.execute("DROP TABLE IF EXISTS joint_mt_mt5_data")
engine.execute("""
CREATE TEMP TABLE joint_mt_mt5_data AS
	WITH positions_filter_mt4 AS (
		SELECT positionid
		FROM hr_vacancies.mt4_marked_trades
		WHERE 1=1
		AND get_bit("type"::bytea, 0) != 1
	),
	positions_filter_mt5 AS (
		SELECT positionid
		FROM hr_vacancies.mt5_marked_trades
		WHERE 1=1
		AND get_bit("type"::bytea, 0) != 1
	)
	SELECT 
		login,
		positionid,
		cmd,
		symbol,
		open_time::timestamp AS open_time,
		close_time::timestamp AS close_time
	FROM hr_vacancies.mt4_trades AS a
	JOIN positions_filter_mt4 AS b
	ON a.ticket = b.positionid
	UNION ALL
	SELECT 
		login,
		a.positionid,
		"action" AS cmd,
		symbol,
		min(CASE WHEN "action" = '0' THEN time::timestamp ELSE '2000-01-01'::date END) AS open_time,
		max(CASE WHEN "action" = '1' THEN time::timestamp ELSE '2999-01-01'::date END) AS close_time
	FROM hr_vacancies.mt5_deals AS a
	JOIN positions_filter_mt5 AS b
	ON a.positionid = b.positionid
	WHERE 1=1
	AND "entry" IN ('0', '1')
	GROUP BY 1,2,3,4
""")

### 2. Задание 1

Посчитать метрики на каждого пользователя:
1. Число сделок, у которых с момента открытия до момента полного закрытия прошло меньше одной минуты
2. Число пар сделок, которые удовлетворяют условиям:
    - Сделки совершены этим пользователем
    - Разница между временем их открытия не более 30 секунд
    - Направление этих сделок – противоположное (одна на покупку, другая на продажу)

In [6]:
SQL_1 = """
WITH metric_1_cte AS (
	SELECT 
		login, 
		sum(CASE WHEN close_time - open_time < INTERVAL '1 minute' THEN 1 ELSE 0 END) AS n_quick_positions -- число быстрых сделок (с открытия до полного закрытия < 1 минуты)
	FROM joint_mt_mt5_data
	WHERE 1=1
	GROUP BY 1
),
metric_2_cte AS (
	SELECT 
		a.login,
		count(DISTINCT GREATEST(a.positionid, b.positionid) || '_' || LEAST(a.positionid, b.positionid)) AS n_position_pairs
	FROM joint_mt_mt5_data AS a
	JOIN joint_mt_mt5_data AS b -- почти кросс-джойн; потенциальная отпимизация: джойнить по отдельности каждого пользователя в цикле
	ON a.login = b.login
	WHERE 1=1
	AND a.positionid != b.positionid
	AND GREATEST(a.open_time, b.open_time) - LEAST(a.open_time, b.open_time) < INTERVAL '30 seconds' -- разница между временем открытия сделок не более 30 секунд
	AND a.cmd != b.cmd -- направление этих сделок – противоположное
	GROUP BY 1
)
SELECT
	a.login, 
	a.n_quick_positions AS metric_1,
	COALESCE(b.n_position_pairs, 0) AS metric_2
FROM metric_1_cte AS a
LEFT JOIN metric_2_cte AS b
ON a.login = b.login
"""

In [7]:
df1 = pd.read_sql_query(SQL_1, con=engine)

In [8]:
df1.shape

(399, 3)

In [9]:
df1.head()

,login,metric_1,metric_2
0,290455359,0,3
1,290465121,5,1
2,290533431,1,1
3,290566631,0,1
4,290586134,2,1


### 3. Задание 2

Найти все возможные пары пользователей, у которых более 10 сделок удовлетворяют условиям:
- Открытия сделок попали в один промежуток времени
- Они открыты по одному и тому же инструменту
- Они принадлежат разным пользователям (одна одному пользователю из пары, другая другому)
- Они открыты в разных направлениях (одна на покупку, другая на продажу)

In [10]:
SQL_2 = """
WITH metric_3_cte AS (
	SELECT DISTINCT
		GREATEST(a.login, b.login) AS user_1,
		LEAST(a.login, b.login) AS user_2,
		GREATEST(a.positionid, b.positionid) AS position_1,
		LEAST(a.positionid, b.positionid) AS position_2
	FROM joint_mt_mt5_data AS a
	CROSS JOIN joint_mt_mt5_data AS b -- данных мало, поэтому кросс-джойн - ОК; потенциальная отпимизация: джойнить по отдельности каждого пользователя в цикле 
	WHERE 1=1
	AND a.login != b.login -- принадлежат разным пользователям
	AND a.symbol = b.symbol -- открыты по одному и тому же инструменту
	AND a.cmd != b.cmd -- открыты в разных направлениях (одна на покупку, другая на продажу)
	AND date_bin('30 seconds', a.open_time, '2000-01-01'::date) = date_bin('30 seconds', b.open_time, '2000-01-01'::date) -- открытия сделок попали в один промежуток времени
)
SELECT user_1, user_2
FROM metric_3_cte
GROUP BY 1,2
HAVING count(*) > 10
"""

In [11]:
df2 = pd.read_sql_query(SQL_2, con=engine)

In [12]:
df2.shape

(14, 2)

In [13]:
df2.head()

,user_1,user_2
0,290675276,21929150
1,290465121,18673759
2,290622568,22198100
3,290675276,22595940
4,290665483,22571075


In [14]:
Jaal(df2.rename(columns={'user_1': 'from', 'user_2': 'to'})).plot()

Parsing the data...Done


### 4. Сохраняем данные в .csv

In [15]:
df1.to_csv('task_1.csv', index=False)
df2.to_csv('task_2.csv', index=False)

No trigger
